In [415]:
import yfinance as yf
import psycopg2 as pg2
import pandas as pd
import pandas_market_calendars as mcal
import traceback
import sys
import os
from datetime import date, timedelta

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'src')))
import database as db

nyse = mcal.get_calendar("NYSE")




In [416]:
def get_dates_for_ticker(ticker):
    """
    This method retrieves a list of all the tickers in the database.

    Returns:
        list: A list of all the tickers in the database.
    """

    conn = db.get_conn() # returns existing db connection, or creates one if needed

    with conn.cursor() as curs:
        try:
            curs.execute(f"SELECT ticker, date, time_of_report FROM earnings_reports WHERE ticker = '{ticker}'")
            dates = curs.fetchall()
        except Exception as e:
            print(f"Failed to get dates from database for ticker {ticker}", traceback.format_exc())
            raise e

    print(f"Retrieved {len(dates)} dates for ticker {ticker}")

    return dates

In [483]:
def populate_prices(ticker):
    """
        Populates relative stock prices for all earnings report records for a given ticker
        This is the root function called which handles every step to update all records for a given ticker

        1. Retrieve all earning report dates for the ticker [get_dates_for_ticker(ticker)]
        2. for each ER date:
            3. find all relative dates for the given date(-30d, +5d, etc) [find_relative_dates(report_date)]
            4. insert stock price data(open, close, high, etc) for each relative date into DB [insert_rel_dates]:
                5. insert_rel_dates first calls get_prices_for_dates(ticker, dates) to get prices at each date
                6. then it will call insert_prices(price_data) where it inserts all prices for each date into the DB
    """
        
    dates = get_dates_for_ticker(ticker)
    # dates includes the date of each earnings report for the given ticker
    # for each date in dates, find all 'relative_dates'. 
    # add the closing price for each relative_date to the database WHERE ticker=ticker AND date=date (undo the -1 day for pre-market!)
    # column names will be minus_1_day, minus_5_day, plus_1_day, plus_30_day, etc...

    
    #below is just for logging/debugging  
    for d in dates:
        rel_dates = find_relative_dates(d[1])
        insert_rel_dates(d, rel_dates)
    
    return rel_dates

In [479]:
def find_relative_dates(date):
    offsets = [1, 2, 3, 4, 5, 10, 20, 30]
    rel_dates = []
    end_date = date + timedelta(days=55)
    sched = nyse.schedule(start_date=date.strftime("%Y-%m-%d"), end_date=end_date.strftime("%Y-%m-%d"))
    
    for o in offsets:
        odate = sched.iloc[o].iloc[0].date()
        rel_dates.append((o, odate))

    
    #REVERSE!
    offsets = [-1, -2, -3, -4, -5, -10, -20, -30]
    start_date = date - timedelta(days=50)
    sched = nyse.schedule(start_date=start_date.strftime("%Y-%m-%d"), end_date=date.strftime("%Y-%m-%d"))

    for o in offsets:
        odate = sched.iloc[o].iloc[0].date()
        rel_dates.append((o, odate))


    #sched[1] is the day after 'date'. For post-market this makes sense. for pre-market we set 'date' back by 1 day, so sched[1] is
    #actually the same day as the report, which is what we want because thats the first trading session 'after' the report
    
    #sched.iloc[-1].iloc[0].date() is equal to date. The [-1] element is the original date passed into this func.
    #that corresponds to the -1 offset, because if it was an after-hours report then the last day before the report
    #IS the same day. If it was a pre-market

    return rel_dates

In [487]:
def get_prices_for_dates(ticker, dates, report_date):
    print(f"\n\n[get_prices_for_dates] ticker={ticker}, len(dates)={len(dates)}. report_date={report_date}")
    yt = yf.Ticker(ticker)
    print("start_date=", dates[0][1].strftime("%Y-%m-%d"))
    print("end_date=", dates[-1][1].strftime("%Y-%m-%d"))
    print("report_date=", report_date)

    # yt.history end date isn't inclusive, have to increase our last date by 1 day to include it
    end_date = dates[-1][1] + timedelta(days=1)
    prices = yt.history(start=dates[0][1].strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    prices.index = prices.index.map(lambda d: d.date().strftime("%Y-%m-%d"))
    price_data = []
    
    for d in dates:
        d_string = d[1].strftime("%Y-%m-%d")
        price_point = {
            "ticker": ticker,
            "report_date": report_date,
            "date": d[1],
            "open": int(round(prices.loc[d_string]['Open'], 2) * 100),
            "close": int(round(prices.loc[d_string]['Close'], 2) * 100),
            "high": int(round(prices.loc[d_string]['High'], 2) * 100),
            "low": int(round(prices.loc[d_string]['Low'], 2) * 100),
            "volume": int(prices.loc[d_string]['Volume'])
        }
        price_data.append(price_point)
        #print(f"[{d[0]} :: {d_string}] {price_point}")

    return price_data
    

In [488]:
def insert_rel_dates(report, rel):
    """
        (realizing this func needs to be renamed... in fact, th
        Takes a single earnings report, and a list of relative dates, and retrieves necessary price data by calling
        get_prices_for_dates. Then inserts all of that data into the database by calling insert_price_data
    """
    dstring = "%Y-%m-%d"
    rel.sort(key=lambda d: d[0])
    #print(f"\nRetrieving and inserting relative date prices for {report}")
    price_data = get_prices_for_dates(report[0], rel, report[1])
    print("PRICE DATA: ", price_data)
    #for r in rel:
        # get closing price for ticker report[0] on r[1]
        #print(f"Updating {report[0]}_{report[1].strftime(dstring)}. {r[0]} days after report({r[1]}) is 420.69")

In [489]:
# Find all earnings reports for 'hut', 
# Find all relative dates for each earnings report(keeping market hours and weekends+holidays in mind),
# Retrieve Open, Close, High, Low, and Volume for each relative day, of each earnings_report,
# Insert all of the above price data into price_history table, alongside the ticker and the original report date
dates = populate_prices("hut") 

Retrieved 12 dates for ticker hut


[get_prices_for_dates] ticker=hut, len(dates)=16. report_date=2024-11-13
start_date= 2024-10-03
end_date= 2024-12-27
report_date= 2024-11-13
PRICE DATA:  [{'ticker': 'hut', 'report_date': datetime.date(2024, 11, 13), 'date': datetime.date(2024, 10, 3), 'open': 1181, 'close': 1173, 'high': 1202, 'low': 1144, 'volume': 2088900}, {'ticker': 'hut', 'report_date': datetime.date(2024, 11, 13), 'date': datetime.date(2024, 10, 17), 'open': 1281, 'close': 1267, 'high': 1336, 'low': 1254, 'volume': 4362600}, {'ticker': 'hut', 'report_date': datetime.date(2024, 11, 13), 'date': datetime.date(2024, 10, 31), 'open': 1803, 'close': 1579, 'high': 1822, 'low': 1578, 'volume': 7379400}, {'ticker': 'hut', 'report_date': datetime.date(2024, 11, 13), 'date': datetime.date(2024, 11, 7), 'open': 1847, 'close': 1897, 'high': 1942, 'low': 1830, 'volume': 6168400}, {'ticker': 'hut', 'report_date': datetime.date(2024, 11, 13), 'date': datetime.date(2024, 11, 8), 'open': 1911,